In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [34]:
class preprocessing_ops():
    def __init__(self, df):
        self.df = df

    def drop_duplicates(self): 
        self.df.drop_duplicates(inplace=True) 
    
    # since WT01 and WT02 both indicate fogs (only difference is if it's heavy or not), we can combine them into one column
    def add_fog(self):
        self.df['WT_FOG'] = np.where((self.df['WT01'] == 1) | (self.df['WT02'] == 1), 1, 0)
        self.df.drop(columns=['WT01','WT02'],inplace=True)

    def remove_cancelled(self):
        self.df = self.df[self.df.CANCELLED != 1]
        self.df.drop(columns=['CANCELLED','CANCELLATION_CODE'],inplace = True)

    def remove_missing_values(self):
        self.df = self.df.loc[:, self.df.isnull().mean() < .9]
    
    # drop IDs, similar columns, post-flight available columns, na values above 35%
    def drop_columns(self):
        self.df.drop(columns=['OP_CARRIER_FL_NUM','Unnamed: 32', 'ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','AIRLINE_ID','OP_UNIQUE_CARRIER','STATION','MANUFACTURE_YEAR'
                         , 'ORIGIN_CITY_NAME','ORIGIN_CITY_NAME_x','ORIGIN_CITY_NAME_y','NAME_x','NAME_y','DEST_CITY_NAME','DEPARTING_AIRPORT','DISPLAY_AIRPORT_NAME','DEST_CITY_NAME'
                         , 'DATE', 'ACTUAL_ELAPSED_TIME','DEP_TIME','DEP_DELAY_NEW', 'ARR_TIME','ARR_DELAY_NEW','ARR_TIME_BLK'
                         , 'CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY', 'WESD','PSUN','TSUN','SN32','SX32','TOBS','WT11','PGTM','SNWD','SNOW'
                         ] ,inplace = True)
    
    # WT has 1 unique value (1 if true and nan if not so filling nans with 0s). then deal with high percentage 0 columns
    def fill_weather_codes(self):
        self.df.fillna({'WT03':'0', 'WT04':'0', 'WT05':'0', 'WT06':'0', 'WT07':'0', 'WT08':'0', 'WT09':'0', 'WT10':'0'}, inplace=True)
        self.df.drop(columns=['WT10','WT07','WT05','WT09','WT04','WT06'],inplace=True)
        self.df['WT03'] = self.df['WT03'].astype(float)
        self.df['WT08'] = self.df['WT08'].astype(float)

    # preserve columns values with more than 2% percentual of delayed flights from the ORIGIN column and replace all other values with 'OTHER'
    def encode_origin_destination(self):
        self.df['ORIGIN'] = np.where(self.df['ORIGIN'].isin(self.df['ORIGIN'].value_counts(normalize=True).loc[lambda x: x > 0.02].index), self.df['ORIGIN'], 'OTHER')
        self.df['DEST'] = np.where(self.df['DEST'].isin(self.df['DEST'].value_counts(normalize=True).loc[lambda x: x > 0.02].index), self.df['DEST'], 'OTHER')

    # delay counts per departure time block and origin
    def delay_blocks_origin(self):
        self.df['DEL_BLOCK_COUNT'] = self.df.groupby('DEP_TIME_BLK')['DEP_DEL15'].transform('sum')
        self.df['DEL_ORIGIN_COUNT'] = self.df.groupby('ORIGIN')['DEP_DEL15'].transform('sum')

    # ‘any’ : If any NA values are present, drop that row or column.
    def drop_na(self):
        self.df.dropna(subset=['TAVG','NUMBER_OF_SEATS','WDF5','WSF5','PRCP','TMIN','TMAX','AWND','WSF2','WDF2'], how='any',inplace=True)

    def encode_string_cols(self):
        string_cols = self.df.select_dtypes(include=['object']).columns
        for col in string_cols:
            label_encoder = LabelEncoder()
            self.df[col] = label_encoder.fit_transform(self.df[col])

    def get_df(self):
        return self.df   

In [52]:
# read
df = pd.read_csv('data/5guys_flight_data.csv')

# instantiate
preprocessing = preprocessing_ops(df)

# call methods
preprocessing.drop_duplicates()
preprocessing.drop_columns()
preprocessing.remove_cancelled()
preprocessing.add_fog()
preprocessing.fill_weather_codes()
preprocessing.drop_na()
preprocessing.encode_origin_destination()
preprocessing.delay_blocks_origin()
preprocessing.encode_string_cols()

pre_df = preprocessing.get_df()

/var/folders/nv/9r2j07yj4w7gpdqz5zp13j3m0000gn/T/ipykernel_23330/2033619164.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/5guys_flight_data.csv')
/var/folders/nv/9r2j07yj4w7gpdqz5zp13j3m0000gn/T/ipykernel_23330/2701430221.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.drop(columns=['CANCELLED','CANCELLATION_CODE'],inplace = True)
/var/folders/nv/9r2j07yj4w7gpdqz5zp13j3m0000gn/T/ipykernel_23330/2701430221.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[

In [53]:
pre_df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,...,TMIN,WDF2,WDF5,WSF2,WSF5,WT03,WT08,WT_FOG,DEL_BLOCK_COUNT,DEL_ORIGIN_COUNT
0,1,28,1,182,14,12,1028,1.0,5,1306,...,8.0,300.0,290.0,25.9,35.1,0.0,1.0,1,48201.0,68867.0
2,1,28,1,2365,14,3,1011,1.0,5,1200,...,8.0,300.0,290.0,25.9,35.1,0.0,1.0,1,48201.0,68867.0
3,1,28,1,1481,14,6,600,1.0,1,853,...,8.0,300.0,290.0,25.9,35.1,0.0,1.0,1,24814.0,68867.0
4,1,28,1,2401,14,3,851,0.0,3,1040,...,8.0,300.0,290.0,25.9,35.1,0.0,1.0,1,42618.0,68867.0
5,1,28,1,2941,14,12,754,1.0,2,1435,...,8.0,300.0,290.0,25.9,35.1,0.0,1.0,1,33240.0,68867.0


In [54]:
pre_df.to_csv('data/5guys_flight_data_preprocessed.csv', index=False)